In [8]:
#Step 1: Scraping Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from selenium import webdriver

#Step 2: Database Dependencies
import pymongo

#Python Dependencies
import pandas as pd


# STEP 1: Web Scraping

### 1A. Use Splinter to create a browser object connected to NASA Mars website

In [9]:
#define path to chrome driver
executable_path = {'executable_path': '/chromedriver'}

#Set default behaviors
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")

#Create browser object
browser = Browser('chrome', **executable_path, headless=False)

#Connect to NASA Mars webpage
browser.visit("https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest")

### 1B. Use BeautifulSoup to parse HTML and retrieve information

In [13]:
#Create a soup object 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [29]:
#Access the titles and article teasers for each news article
articles = soup.find_all("li", class_="slide")
titles = []
teasers = []

for article in articles:
    title = article.find('div', class_="content_title").text
    teaser = article.find('a').text
    titles.append(title)
    teasers.append(teaser)


In [102]:
rows = soup.find_all("div", class_="article_teaser_body")

teaser = []
for row in rows:
    body = row.next.strip()
    teaser.append(body)
teaser

[]

# Step2: MongoDB and Flask Application

In [85]:

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)